<a href="https://colab.research.google.com/github/arpitkj/Movie-recommender/blob/new_branch/Movie_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing libraries
import numpy as np
import pandas as pd

In [3]:
#reading files from Movielens dataset
data = pd.io.parsers.read_csv('/content/drive/MyDrive/movie recommender/ml-1m/ratings.dat', names=['user_id', 'movie_id', 'rating', 'time'], engine='python', delimiter='::')

movie_data = pd.io.parsers.read_csv('/content/drive/MyDrive/movie recommender/ml-1m/movies.dat',names=['movie_id', 'title', 'genre'],engine='python', delimiter='::')

In [10]:
data.head()

,user_id,movie_id,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
movie_data.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
#Creating the ratings matrix with rows as movies and columns as users
ratings_mat = np.ndarray(shape=(np.max(data.movie_id.values), np.max(data.user_id.values)),dtype=np.uint8)

ratings_mat[data.movie_id.values-1, data.user_id.values-1] = data.rating.values

In [5]:
#normalising matrix
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

In [6]:
#computing svd
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [7]:
#calculating cosine similarity and returning top n results
def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = data[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Helper function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.movie_id == movie_id].title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.movie_id == id].title.values[0])

In [9]:
#Select  principal components to represent the movies, a movie_id to find recommendations and print the top_n results.
k = 50
movie_id = 2 # Grab an id from movies.dat
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)
print_similar_movies(movie_data, movie_id, indexes)


Recommendations for Jumanji (1995): 

Jumanji (1995)
Indian in the Cupboard, The (1995)
NeverEnding Story III, The (1994)
Cutthroat Island (1995)
Muppet Treasure Island (1996)
Honey, I Shrunk the Kids (1989)
Flubber (1997)
Prince of Egypt, The (1998)
Hercules (1997)
Flight of the Navigator (1986)
